In [2]:
import pymysql as ps
import requests
import time

In [3]:
#db와 연동해서 시작점, 끝점, 허용시간 테이블 받아옴
db = ps.connect(
    host="project-db-stu3.smhrd.com", user="Insa4_IOTA_hacksim_3", passwd="aishcool3", db="Insa4_IOTA_hacksim_3", charset="utf8mb4",
    port = 3307
)
cursor=db.cursor()
sql='select start_p from street_park'
cursor.execute(sql)
start_street_data=cursor.fetchall()
cursor=db.cursor()
sql='select end_p from street_park'
cursor.execute(sql)
end_street_data=cursor.fetchall()
cursor=db.cursor()
sql='select street_id, permit_t from street_park'
cursor.execute(sql)
permit_time=cursor.fetchall()

In [4]:
#시작점, 끝점 gps좌표를 담아둘 리스트
street_start_point=[]
street_end_point=[]
#현재 위치가 어느 지점에 속해있는지 확인하는 변수
target_point=0
#gps 모듈에서 받아온 값을 담아둘 변수
longitude_now=0
latitude_now=0

In [5]:
#카카오맵 api를 이용해서 도로명주소를 위도,경도로 변환하는 함수
def address_to_gps_coordinates_kakao(address, api_key):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {
        "Authorization": f"KakaoAK {api_key}"
    }
    params = {
        "query": address
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    if data.get("documents"):
        first_result = data["documents"][0]
        latitude = first_result["y"]
        longitude = first_result["x"]
        return float(latitude), float(longitude)
    else:
        return None
# 카카오 API 키를 발급받은 후 아래에 적절한 값을 넣어주세요
kakao_api_key = "2423ceacbff8cf8f6ed1aa8bd18a242b"

#현재 gps좌표가 어느 구간에 있는지 확인하는 함수
def is_inside_rectangle(point1, point2, target_point):
    min_x = min(point1[0], point2[0])
    max_x = max(point1[0], point2[0])
    min_y = min(point1[1], point2[1])
    max_y = max(point1[1], point2[1])

    if min_x <= target_point[0] <= max_x and min_y <= target_point[1] <= max_y:
        return True
    return False


In [6]:
#현재 gps 모듈이 없어서 임의의 gps값 설정
longitude_now=126.8398
latitude_now=35.1470000

In [7]:
#시작점 변환 후 담아주기
addresses = start_street_data
for address in addresses:
    coordinates = address_to_gps_coordinates_kakao(address, kakao_api_key)
    if coordinates:
        latitude, longitude = coordinates
        street_start_point.append((longitude,latitude))

In [8]:
#끝점 변환 후 담아주기
addresses = end_street_data
for address in addresses:
    coordinates = address_to_gps_coordinates_kakao(address, kakao_api_key)
    if coordinates:
        latitude, longitude = coordinates
        street_end_point.append((longitude,latitude))

In [9]:
#현재 gps좌표가 어느 노상주차 허용구간에 있는지 확인
for i in range(len(street_start_point)):
    start_point = street_start_point[i]
    end_point = street_end_point[i]
    now_point = (longitude_now,latitude_now)
    if is_inside_rectangle(start_point, end_point, now_point):
        target_point+=i+1

In [10]:
#gps좌표 index로 허용시간 찾기
cursor=db.cursor()
sql=f'select street_id, permit_t from street_park where street_id={target_point}'
cursor.execute(sql)
permit_time_here=cursor.fetchall()

In [11]:
#현재 시간 받아서 now 변수에 담아주기.
now=time
now_hour=now.localtime().tm_hour
print(now_hour)

16


In [12]:
print(permit_time_here)

((3, '06:00~22:00'),)


In [13]:
permit_time_here[0][1]

'06:00~22:00'

In [14]:
if len(permit_time_here[0][1])>12:
    if now_hour>=int(permit_time_here[0][1][13:15]) or now_hour<=int(permit_time_here[0][1][19:21]) :
        print('주차가 가능한 시간입니다. led on',end=' ')
    else :
        print('주차가 가능한 시간이 아닙니다. led off',end=' ')
if int(permit_time_here[0][1][:2])<int(permit_time[0][1][6:8]):
    if now_hour>=int(permit_time_here[0][1][:2]) and now_hour<=int(permit_time_here[0][1][6:8]) :
        print('주차가 가능한 시간입니다. led on')
    else :
        print('주차가 가능한 시간이 아닙니다. led off')
else :
    if now_hour>=int(permit_time_here[0][1][:2]) or now_hour<=int(permit_time_here[0][1][6:8]) :
        print('주차가 가능한 시간입니다. led on')
    else :
        print('주차가 가능한 시간이 아닙니다. led off')

주차가 가능한 시간입니다. led on


In [40]:
#끝내기전에 db와 연결종료
db.commit()
db.close()

InterfaceError: (0, '')